In [1]:
#!pip install flask

In [2]:
#!pip install flask_restful

In [3]:
from flask import Flask
from flask_restful import Resource, Api, reqparse
import pandas as pd
import ast

In [4]:
app = Flask(__name__)
api = Api(app)

In [5]:
class Users(Resource):
    pass
    
class Locations(Resource):
    pass

In [6]:
class Users(Resource):
    def get(self):
        data = pd.read_csv("C:/Users/santy/Desktop/API/users.csv")
        data = data.to_dict()  # convert dataframe to dict
        return {'data': data}, 200

    def post(self):
        parser = reqparse.RequestParser()  # initialize
        parser.add_argument('userId', required=True)  # add args
        parser.add_argument('name', required=True)
        parser.add_argument('city', required=True)
        args = parser.parse_args()  # parse arguments to dictionary

        # read our CSV
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')

        if args['userId'] in list(data['userId']):
            return {
                'message': f"'{args['userId']}' already exists."
            }, 409
        else:
            # create new dataframe for new values
            new_data = pd.DataFrame({
                'userId': [args['userId']],
                'name': [args['name']],
                'city': [args['city']],
                'locations': [[]]
            })
            # add the new values
            data = data.append(new_data, ignore_index=True)
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()}, 200
    
    def put(self):
        parser = reqparse.RequestParser()
        parser.add_argument('userId', required=True)
        parser.add_argument('location', required=True)
        args = parser.parse_args()
        
        #read our data
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        
        if args['userId'] in list(data['userId']):
            data['locations'] = data['locations'].apply(
                lambda x:ast.literal_eval(x)
            )
            
            #select our user
            user_data = data[data['userId'] == args['userId']]
            #update location
            user_data['locations'] = user_data['locations'].values[0] \
                .append(args['location'])
        
            #write to csv
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()}, 200
        else:
            return {'message': f"'{args['userId']}' user not found."}, 404
        
    def delete(self):
        parser = reqparse.RequestParser()
        parser.add_argument('userId', required=True)
        args = parser.parse_args()
        
        #reading data
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        
        if args['userId'] in list(data['userId']):
            data = data[data['userId'] != args['userId']]
            
            #save to csv
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()}, 200
        else:
            return {'message': f"'{args['userId']}' user not found."}, 404

class Locations(Resource):
    def get(self):
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        return {'data': data.to_dict()}, 200
    
    def post(self):
        parser = reqparse.RequestParser()
        parser.add_argument('locationId', required=True, type=int)
        parser.add_argument('name', required=True)
        parser.add_argument('rating', required=True)
        args = parser.parse_args()
        
        #read data
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        
        #check if location already exists
        if args['locationId'] in list(data['locationId']):
            return {'message':f"'{args['locationId']}'already exists."}, 409
        else:
            new_data = pd.DataFrame({
                'locationId': [args['locationId']],
                'name': [args['name']],
                'rating': [args['rating']]
            })
            data = data.append(new_data, ignore_index=True)
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()}, 200
    
    def patch(self):
        parser = reqparse.RequestParser()
        parser.add_argument('locationId', required=True, type=int)
        parser.add_argument('name', required=True)
        parser.add_argument('rating', required=True)
        args = parser.parse_args()
        
         #read data
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        
        #chech location exist already
        if args['locationId'] in list(data['locationId']):
            user_data = data[data['locationId'] == args['locationId']]
        
            #update name
            if 'name' in args:
                user_data['name'] = args['name']
        
            #update rating
            if 'rating' in args:
                user_data['rating'] = args['rating']
            
            #update data
            data[data['locationId'] == args['locationId']] = user_data
            #save data
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()},200
        else:
            return{'message':f"'{args['locationId']}'location does not exist."}, 404
    
    def delete(self):
        parser = reqparse.RequestParse()
        parser.add_argument('locationId', required=True, type=int)
        args = parser.parse_args()
        
        #read data
        data = pd.read_csv('C:/Users/santy/Desktop/API/users.csv')
        
        #check location exists already
        if args['locationId'] in list(data['locationId']):
            #if it exists we delete it
            data = data[data['locationId'] != args['locationId']]
            #save data
            data.to_csv('C:/Users/santy/Desktop/API/users.csv', index=False)
            return {'data': data.to_dict()},200
        else:
            return {'message': f"'{args['locationId']}' location does not exist."}
            
            

In [7]:
api.add_resource(Users, '/users')  # add endpoints
api.add_resource(Locations, '/locations')

In [ ]:
if __name__ == '__main__':
    app.run()  # run our Flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2021 12:14:28] "GET / HTTP/1.1" 404 -
